In [1]:
import json 
import pandas as pd
import snowflake.connector as sf
import os
import xlwings as xw
import glob
pd.options.mode.chained_assignment = None
def fill_data_if_dup_column(column_list_to_fill,df_raw):
    for column_need_to_fill in column_list_to_fill:
        column_contain_list = []
        for column in df_raw.columns:
            if column_need_to_fill in column:
                column_contain_list.append(column)
        if len(column_contain_list) > 1 :
            # coalesce if many column dup name
            df_raw[column_need_to_fill] = df_raw[column_contain_list].bfill(axis=1).iloc[:, 0]
    return df_raw

In [4]:
path_csv_raw = "D:\python\cl_nonedi\putput_csv_raw"
os.chdir(path_csv_raw)
csv_list = glob.glob('*.csv', recursive=True)
columns_dict = {}
for file in csv_list:
    columns_dict[file] =  pd.read_csv(file).columns.to_list()

In [ ]:
df_manual_fixing = pd.read_excel(r'D:\python\cl_nonedi\manual_fixing.xlsx',sheet_name='Error_header_1').reset_index(drop=True).map(lambda x: str(x).replace('nan',''))
manual_fixing_dict = df_manual_fixing.to_dict(orient = 'index')
manual_fixing_dict

In [6]:
# columns_dict_test = {'0650_21225598TIF_liq.csv' : columns_dict['0650_21225598TIF_liq.csv']}
columns_dict_fixed = {}
for file,column_list in columns_dict.items():
    column_list_fixed = []
    for column_name in column_list:
        for value in list(manual_fixing_dict.values()):
            if value['Error'] in column_name:
                column_name = column_name.replace(value['Error'],value['Error_fix'])
        column_list_fixed.append(column_name.upper().strip().replace(' ',''))
    columns_dict_fixed[file] = column_list_fixed
columns_dict_fixed

{'0650_21225598TIF_liq.csv': ['INDEX',
  'FILENAME',
  'KV-TAXINVOICE',
  'KV-INVOICEAND',
  'KV-VENDOR',
  'KV-GST',
  'RODUCT',
  'PRODUCTDESCRIPTION',
  'UM',
  'QTY',
  'UNITVALUE',
  'PRODUCTVALUE',
  'DISCOUNT',
  'CONTAINERDEPOSIT',
  'WET',
  'FREIGHT',
  'HANDLINGADJUSTMENT',
  'LUCEXCLGST',
  'LINEVALUEEXCLGST',
  'KV-INVOICETOTALINCLGST',
  'KV-TOTALLITRES',
  'KV-TOTALCARTONS',
  'KV-SUBTOTAL',
  'KV-ORDERNUMBERTOTAL',
  'F-DESCRIPTION',
  'F-VENDORTAXID'],
 '0650_21226112TIF_liq.csv': ['INDEX',
  'FILENAME',
  'KV-TAXINVOICE',
  'KV-VENDOR',
  'KV-ADJUSTMENTLUCEXCLGST',
  'KV-LINEVALUEEXCLGST',
  'KV-FORCREDITCARDANDDIRECTDEBITINCLGST',
  'PRODUCT',
  'PRODUCTDESCRIPTION',
  'UM',
  'QTY',
  'UNITVALUE',
  'PRODUCTVALUE',
  'DISCOUNT',
  'CONTAINERDEPOSIT',
  'WET',
  'FREIGHT',
  'HANDLINGADJUSTMENT',
  'LUCEXCLGST',
  'LINEVALUEEXCLGST',
  'KV-INVOICETOTAL',
  'KV-TOTALLITRES',
  'KV-TOTAL',
  'F-QUANTITY',
  'F-TOTALTAX',
  'F-VENDORTAXID'],
 '0650_21226594TIF_liq.csv':

In [7]:
df_manual_fixing_2 = pd.read_excel(r'D:\python\cl_nonedi\manual_fixing.xlsx',sheet_name='Error_header_2').reset_index(drop=True).map(lambda x: str(x).replace('nan',''))
manual_fixing_dict_2 = df_manual_fixing_2.to_dict(orient = 'index')
manual_fixing_dict_2

{0: {'Error': 'PRODUCT', 'Error_fix': 'PRODUCT_ID'},
 1: {'Error': 'IODUCT', 'Error_fix': 'PRODUCT_ID'},
 2: {'Error': 'ODUCT', 'Error_fix': 'PRODUCT_ID'},
 3: {'Error': 'RODUCT', 'Error_fix': 'PRODUCT_ID'},
 4: {'Error': 'PODUCT', 'Error_fix': 'PRODUCT_ID'},
 5: {'Error': 'PRODUCTDESCRIPTION', 'Error_fix': 'PRODUCT_NAME'},
 6: {'Error': 'EXCLGSTLUC', 'Error_fix': 'INV_PRICE'},
 7: {'Error': 'LUCEXCLGST', 'Error_fix': 'INV_PRICE'},
 8: {'Error': 'LINEVALUEEXCLGST', 'Error_fix': 'LINE_VALUE'},
 9: {'Error': 'LINEVALUEEXCLLGST', 'Error_fix': 'LINE_VALUE'},
 10: {'Error': 'QTY', 'Error_fix': 'INV_QTY'},
 11: {'Error': 'INDEX', 'Error_fix': 'INDEX_CSV'}}

In [ ]:
columns_dict_fixed_2 = {}
for file,column_list in columns_dict_fixed.items():
    column_list_fixed = []
    for column_name in column_list:
        for value in list(manual_fixing_dict_2.values()):
            if value['Error'] == column_name:
                column_name = column_name.replace(value['Error'],value['Error_fix'])
                break
        #check dup column
        if column_name in column_list_fixed:
            i = 1
            for column_fixed in column_list_fixed:
                if column_name == column_fixed:
                    i+= 1
            column_name += str(i)
        column_list_fixed.append(column_name)
    columns_dict_fixed_2[file] = column_list_fixed
columns_dict_fixed_2

In [ ]:
df_all = pd.DataFrame()
for file in csv_list:
    df = pd.read_csv(file)
    df.columns = columns_dict_fixed_2[file]
    df = fill_data_if_dup_column(column_list_to_fill = ['PRODUCT_ID','PRODUCT_NAME','INV_QTY','INV_PRICE','LINE_VALUE'],df_raw = df)
    print(f'----------------------------------{file}---------------')
    print(df)
    df_all = pd.concat([df_all.reset_index(drop=True), df.reset_index(drop=True)])

In [ ]:
df_test = df_all
df_test['DATA'] = df_test.drop(columns=['INDEX_CSV', 'FILENAME', 'PRODUCT_ID','PRODUCT_NAME','INV_QTY','INV_PRICE','LINE_VALUE']).applymap(str).apply(lambda x: json.dumps(x.to_dict()), axis=1)
# df_test = df_test[['INDEX_CSV', 'FILENAME', 'PRODUCT_ID','PRODUCT_NAME','INV_QTY','INV_PRICE','LINE_VALUE','DATA']].apply(lambda x : x.replace('nan',''))
# df_test = df_test.astype({'PRODUCT_ID': 'str','INV_QTY':'int','INV_PRICE':'float','LINE_VALUE':'float'})
# df_test


In [ ]:
df_test = df_test[['INDEX_CSV', 'FILENAME', 'PRODUCT_ID','PRODUCT_NAME','INV_QTY','INV_PRICE','LINE_VALUE']]
df_test

In [ ]:
df_manual_fixing_3 = pd.read_excel(r'D:\python\cl_nonedi\manual_fixing.xlsx',sheet_name='Error_detail').reset_index(drop=True).map(lambda x: str(x).replace('nan',''))
manual_fixing_dict_3 = df_manual_fixing_3.to_dict(orient = 'index')
manual_fixing_dict_3

In [69]:
list_dict_test = df_test.to_dict(orient='records')
#clean itemid & itemname
for i in range(len(list_dict_test)):
    try:
        list_dict_test[i]['PRODUCT_ID'] = int(list_dict_test[i]['PRODUCT_ID'])
    except:
        list_digital = []
        list_notdigital = []
        for char in str(list_dict_test[i]['PRODUCT_ID']):
            if char.isdigit():
                list_digital.append(char)
            else:
                list_notdigital.append(char)                  
        list_dict_test[i]['PRODUCT_ID'] = ''.join(list_digital)
        list_dict_test[i]['PRODUCT_NAME'] = ''.join(list_notdigital) + ' ' + str(list_dict_test[i]['PRODUCT_NAME'])
        for value in manual_fixing_dict_3.values():
            if value['Error'] in list_dict_test[i]['PRODUCT_NAME']:
                list_dict_test[i]['PRODUCT_NAME'] = list_dict_test[i]['PRODUCT_NAME'].replace(value['Error'],value['Error_fix'])
    # clean QTY
    try:
        list_dict_test[i]['INV_QTY'] = abs(int(list_dict_test[i]['INV_QTY']))
    except:
        list_digital = []
        list_notdigital = []
        for char in str(list_dict_test[i]['INV_QTY']):
            if char.isdigit():
                list_digital.append(char)
            else:
                list_notdigital.append(char)     
        try:             
            list_dict_test[i]['INV_QTY'] = abs(int(''.join(list_digital)))
        except ValueError :
            list_dict_test[i]['INV_QTY'] = ''.join(list_digital)
    # clean LINE_VALUE
    try:
        list_dict_test[i]['LINE_VALUE'] = abs(float(list_dict_test[i]['LINE_VALUE']))
    except:
        # exxist character not digital
        list_dict_test[i]['LINE_VALUE'] = list_dict_test[i]['LINE_VALUE'].replace(',','.')
        list_digital = []
        list_notdigital = []
        for char in str(list_dict_test[i]['LINE_VALUE']) or char == '.':
            if char.isdigit() or char == '.':
                list_digital.append(char)
            else:
                list_notdigital.append(char)   
        #after remove notdigital some number may be .23.56. -> remove 2  character . 
        try:             
            list_dict_test[i]['LINE_VALUE'] = abs(float(''.join(list_digital)))
        except ValueError :
            if list_digital == []:
                list_dict_test[i]['LINE_VALUE'] = ''
            else:
                if list_digital[0] == '.':
                    list_digital.pop(0)
                if list_digital[-1] == '.':
                    list_digital.pop(-1)
                list_dict_test[i]['LINE_VALUE'] = abs(float(''.join(list_digital)))
    # clean LUC_VALUE
    try:
        list_dict_test[i]['INV_PRICE'] = abs(float(list_dict_test[i]['INV_PRICE']))
    except:
        # exxist character not digital
        list_dict_test[i]['INV_PRICE'] = list_dict_test[i]['INV_PRICE'].replace(',','.')
        list_digital = []
        list_notdigital = []
        for char in str(list_dict_test[i]['INV_PRICE']) or char == '.':
            if char.isdigit() or char == '.':
                list_digital.append(char)
            else:
                list_notdigital.append(char)   
        #after remove notdigital some number may be .23.56. -> remove 2  character . 
        try:             
            list_dict_test[i]['INV_PRICE'] = abs(float(''.join(list_digital)))
        except ValueError :
            if list_digital == []:
                list_dict_test[i]['INV_PRICE'] = ''
            else:
                if list_digital[0] == '.':
                    list_digital.pop(0)
                if list_digital[-1] == '.':
                    list_digital.pop(-1)
                list_dict_test[i]['INV_PRICE'] = abs(float(''.join(list_digital)))
    try :
        list_dict_test[i]['INV_PRICE_2'] = round(float(list_dict_test[i]['LINE_VALUE']) / int(list_dict_test[i]['INV_QTY']),2)
    except:
        list_dict_test[i]['INV_PRICE_2'] = ''
    try :
        list_dict_test[i]['INV_PRICE'] = round(float(list_dict_test[i]['INV_PRICE']),2)
    except:
        list_dict_test[i]['INV_PRICE'] = ''
    list_dict_test[i]['VEND_NUM'] = list_dict_test[i]['FILENAME'].split('_')[0]
    list_dict_test[i]['REF_NUM'] = list_dict_test[i]['FILENAME'].split('_')[1]
    list_dict_test[i]['LINE_NUM'] = list_dict_test[i]['INDEX_CSV']

df = pd.DataFrame(list_dict_test)
df = df[['LINE_NUM','FILENAME','VEND_NUM','REF_NUM','PRODUCT_ID','PRODUCT_NAME','INV_QTY','INV_PRICE','LINE_VALUE','INV_PRICE_2']]
df.to_csv(r'D:\python\cl_nonedi\test4.csv',index=False)